In [ ]:
import sys
import os
import myFunctions
from myFunctions import install_packages, save_table 
### packages required
install_packages()

### importing required packages
from tabulate import tabulate
import pandas as pd
import optuna
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, roc_auc_score
from torch.utils.data import DataLoader, TensorDataset
import joblib


## Model Classes
# LSTM Model
class LSTMModel(nn.Module):
    """
    LSTM model class for time series forecasting.

    Attributes:
        lstm (nn.LSTM): LSTM layer for sequence processing.
        fc (nn.Linear): Fully connected layer to output predictions.
    """
    def __init__(self, input_size, hidden_size, num_layers, dropout):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, dropout=dropout, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)
    
    def forward(self, x):
        """
        Forward pass through the model.

        Args:
            x (torch.Tensor): Input tensor with shape (batch_size, seq_len, input_size).

        Returns:
            torch.Tensor: Output tensor with predictions.
        """
        lstm_out, (hn, cn) = self.lstm(x)
        out = self.fc(lstm_out[:, -1, :])  # Using the last output of the sequence
        return out

# GRU Model
class GRUModel(nn.Module):
    """
    GRU model class for time series forecasting.

    Attributes:
        gru (nn.GRU): GRU layer for sequence processing.
        fc (nn.Linear): Fully connected layer to output predictions.
    """
    def __init__(self, input_size, hidden_size, num_layers, dropout):
        super(GRUModel, self).__init__()
        self.gru = nn.GRU(input_size, hidden_size, num_layers, dropout=dropout, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)
    
    def forward(self, x):
        """
        Forward pass through the model.

        Args:
            x (torch.Tensor): Input tensor with shape (batch_size, seq_len, input_size).

        Returns:
            torch.Tensor: Output tensor with predictions.
        """
        gru_out, hn = self.gru(x)
        out = self.fc(gru_out[:, -1, :])  # Using the last output of the sequence
        return out

# CNN-LSTM Model
class CNNLSTMModel(nn.Module):
    """
    CNN-LSTM model class combining convolutional layers with LSTM for time series forecasting.

    Attributes:
        conv1d (nn.Conv1d): Convolutional layer for feature extraction.
        lstm (nn.LSTM): LSTM layer for sequence processing.
        fc (nn.Linear): Fully connected layer to output predictions.
    """
    def __init__(self, input_size, hidden_size, num_layers, dropout, conv_filters):
        super(CNNLSTMModel, self).__init__()
        self.conv1d = nn.Conv1d(input_size, conv_filters, kernel_size=3, padding=1)
        self.lstm = nn.LSTM(conv_filters, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        """
        Forward pass through the model.

        Args:
            x (torch.Tensor): Input tensor with shape (batch_size, seq_len, input_size).

        Returns:
            torch.Tensor: Output tensor with predictions.
        """
        x = x.permute(0, 2, 1)  # For Conv1d, we need [batch, channels, seq_len]
        x = self.conv1d(x)
        x = x.permute(0, 2, 1)  # Returning to [batch, seq_len, channels]
        lstm_out, (hn, cn) = self.lstm(x)
        out = self.fc(lstm_out[:, -1, :])  # Using the last output of the sequence
        return out

# CNN-GRU Model
class CNNGRUModel(nn.Module):
    """
    CNN-GRU model class combining convolutional layers with GRU for time series forecasting.

    Attributes:
        conv1d (nn.Conv1d): Convolutional layer for feature extraction.
        gru (nn.GRU): GRU layer for sequence processing.
        fc (nn.Linear): Fully connected layer to output predictions.
    """
    def __init__(self, input_size, hidden_size, num_layers, dropout, conv_filters):
        super(CNNGRUModel, self).__init__()
        self.conv1d = nn.Conv1d(input_size, conv_filters, kernel_size=3, padding=1)
        self.gru = nn.GRU(conv_filters, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        """
        Forward pass through the model.

        Args:
            x (torch.Tensor): Input tensor with shape (batch_size, seq_len, input_size).

        Returns:
            torch.Tensor: Output tensor with predictions.
        """
        x = x.permute(0, 2, 1)  # For Conv1d, we need [batch, channels, seq_len]
        x = self.conv1d(x)
        x = x.permute(0, 2, 1)  # Returning to [batch, seq_len, channels]
        gru_out, hn = self.gru(x)
        out = self.fc(gru_out[:, -1, :])  # Using the last output of the sequence
        return out


def scale_features(df, exclude_columns=['date', 'day']):
    """
    Scales all columns of the DataFrame except the ones specified in exclude_columns.

    Args:
        df (pd.DataFrame): The DataFrame containing the data.
        exclude_columns (list): Columns that should not be scaled.

    Returns:
        pd.DataFrame: Scaled DataFrame.
        dict: A dictionary containing the scalers for each scaled column.
    """
    scalers = {}
    columns_to_scale = [col for col in df.columns if col not in exclude_columns]
    
    df_scaled = df.copy()
    for col in columns_to_scale:
        scaler = MinMaxScaler()
        df_scaled[col] = scaler.fit_transform(df[[col]])
        scalers[col] = scaler
    
    return df_scaled, scalers

def data_to_array(df, window_size, target, features):
    """
    Prepares X and y with targets shifted for the next day after the window.

    Args:
        df (pd.DataFrame): The dataframe containing the data.
        window_size (int): The window size (e.g., 7 days).
        target (str): The target column name (e.g., 'close_price_target').
        features (list): List of feature column names (e.g., ['open', 'high', 'low', 'close']).

    Returns:
        np.ndarray: Input features (X).
        np.ndarray: Target values (y).
        np.ndarray: Dates associated with the targets.
    """
    X = []
    y = []
    y_dates = []

    for i in range(len(df) - window_size):
        # Access the target column directly by its name
        target_value = df.iloc[i + window_size][target]
        y.append(target_value)
        y_dates.append(df.iloc[i + window_size]['date'])
        
        # Prepare the features using the provided column names
        X.append(df.iloc[i:i + window_size][features].values)

    return np.array(X), np.array(y), np.array(y_dates)

def segment_data(df, test_size=0.15):
    """
    Segments the data into training and test sets based on the test size percentage.

    Args:
        df (pd.DataFrame): The DataFrame to be segmented.
        test_size (float): The percentage of data to be used for testing.

    Returns:
        pd.DataFrame: Training data.
        pd.DataFrame: Testing data.
    """
    test_len = int(len(df) * test_size)
    train_data = df[:-test_len]  # 85% for training
    test_data = df[-test_len:]   # 15% for testing
    
    return train_data, test_data


def train_evaluate_model(trial, X_train, y_train, X_test, y_test, target, window_size, look_forward, model_type, study_name, model_dir):
    """
    Trains and evaluates the model using Optuna for hyperparameter tuning.

    Args:
        trial (optuna.Trial): The trial object for hyperparameter tuning.
        X_train (np.ndarray): Training features.
        y_train (np.ndarray): Training target.
        X_test (np.ndarray): Test features.
        y_test (np.ndarray): Test target.
        target (str): The target column.
        window_size (int): The window size for the model.
        look_forward (int): The look-forward period for the prediction.
        model_type (str): The type of model ('LSTM', 'GRU', etc.).
        study_name (str): The study name for saving results.
        model_dir (str): The directory to save model results.

    Returns:
        float: The error value (AUC or MSE).
    """
    X_train = torch.tensor(X_train, dtype=torch.float32)
    y_train = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
    X_test = torch.tensor(X_test, dtype=torch.float32)
    y_test = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

    epochs = trial.suggest_int('epochs', 10, 20)
    batch_size = trial.suggest_categorical('batch_size', [32, 64])
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-2, log=True)
    dropout = trial.suggest_float('dropout', 0.2, 0.5)
    num_layers = trial.suggest_int('num_layers', 2, 3)
    hidden_size = trial.suggest_int('hidden_size', 64, 128)

    if model_type == 'LSTM':
        model = LSTMModel(input_size=X_train.shape[2], hidden_size=hidden_size, num_layers=num_layers, dropout=dropout)
    elif model_type == 'GRU':
        model = GRUModel(input_size=X_train.shape[2], hidden_size=hidden_size, num_layers=num_layers, dropout=dropout)
    elif model_type == 'CNN-LSTM':
        conv_filters = trial.suggest_int('conv_filters', 16, 64)
        model = CNNLSTMModel(input_size=X_train.shape[2], hidden_size=hidden_size, num_layers=num_layers, dropout=dropout, conv_filters=conv_filters)
    else:
        conv_filters = trial.suggest_int('conv_filters', 16, 64)
        model = CNNGRUModel(input_size=X_train.shape[2], hidden_size=hidden_size, num_layers=num_layers, dropout=dropout, conv_filters=conv_filters)

    criterion = nn.MSELoss() if 'price' in target else nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    train_dataset = TensorDataset(X_train, y_train)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
    
    model.eval()
    with torch.no_grad():
        predictions = model(X_test).squeeze().numpy()
        if 'price' in target:
            # MSE for regression
            error = mean_squared_error(y_test.numpy(), predictions)
        else:
            # AUC for classification
            predictions = (predictions > 0.5).astype(int)
            error = roc_auc_score(y_test.numpy(), predictions)
    
    # Saving the model and results
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)

    model_name = f'{model_type}_{study_name}.pth'
    model_path = os.path.join(model_dir, model_name)
    torch.save(model.state_dict(), model_path)
    joblib.dump(study.best_params, os.path.join(model_dir, f'{study_name}_params.pkl'))

    return error
